## ch2. 머신러닝 프로젝트 처음부터 끝까지

### 2.1) 데이터 다운로드

In [ ]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path,"housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

load_housing_data()

### 2.2) 데이터 구조 훑어보기

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
# %matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20, 15))
plt.show()

### 2.3) 테스트 세트 만들기

In [ ]:
import numpy as np

# 완벽하지 않은 테스트 데이터 분할 방식
# 다시 실행할 때마다 다른 테스트 세트 생성됨 => 전체 데이터셋 보는 셈
def split_train_test(data, test_ratio):
    # solution 1. np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)
len(train_set)

In [ ]:
len(test_set)

In [ ]:
from zlib import crc32

# 샘플의 식별자를 사용
# 각 샘플마다 식별자의 해시값 계산, 해시 최댓값의 20%보다 작거나 같은 샘플만 테스트 세트로 보냄.
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2 ** 32

In [ ]:
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
# housing 데이터프레임에 고유 식별자 컬럼 없으므로 고유 식별자 추가
housing_with_id = housing.reset_index()

train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, 'index')

In [ ]:
# 위도, 경도 좌표 값 특성 이용한 id 생성
housing_with_id['id'] = housing['longitude'] * 1000 + housing['latitude']
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, 'id')

In [ ]:
# 사이킷런을 이용한 train, test split
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
# 완전 무작위 샘플링 통한 결과 편향성 방지
# 계층적 샘플링 필요 => 각 계층별 고른 개체 수 확보 필요

housing['income_cat'] = pd.cut(housing['median_income'], 
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

housing['income_cat'].hist()

In [ ]:
# 층화 무작위 추출=> train, test data 나눌 때 안에 계층 비율 유지하도록 함.
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set['income_cat'].value_counts() / len(strat_test_set)

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)

### 2.4) 데이터 이해를 위한 탐색과 시각화

In [ ]:
housing = strat_train_set.copy()

# 지리적 데이터 시각화
housing.plot(kind='scatter', x='longitude', y='latitude')

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4, 
            s=housing['population'] / 100, label='population', figsize=(10, 7),
            c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True)
plt.legend()

# 중간 주택 가격은 인구 밀도와 관련 매우 큼

In [ ]:
# 상관관계 조사 => 데이터셋이 크지 않으므로 표준 상관계수 (피어슨) 사용
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)

In [ ]:
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']

corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

### 2.5) 머신러닝 알고리즘을 위한 데이터 준비

In [ ]:
housing = train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

In [ ]:
# 데이터 정제 => 결측치 처리
# case 1 => 해당 구역 제거
housing.dropna(subset=['total_bedrooms'])

# case 2 => 전체 특성 삭제
housing.drop('total_bedrooms', axis=1)

# case 3 => 평균, 중간값 등으로 채움
median = housing['total_bedrooms'].median()
housing['total_bedrooms'].fillna(median, inplace=True)
housing

In [29]:
# 결측치 대치
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

# 중간값이 수치형 특성에서만 계산될 수 있기 때문에 텍스트 특성 컬럼 제외
housing_num = housing.drop('ocean_proximity', axis=1)

# 훈련 데이터에 적용
imputer.fit(housing_num)

# 각 특성 중간값 계산한 결과 객체의 statistics_ 속성에 저장
imputer.statistics_

# 위 코드와 실행결과 동일
housing_num.median().values

X = imputer.transform(housing_num)

housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
housing_tr

In [30]:
# 텍스트와 범주형 특성 다루기
housing_cat = housing[['ocean_proximity']]
housing_cat.head()

,ocean_proximity
14196,NEAR OCEAN
8267,NEAR OCEAN
17445,NEAR OCEAN
14265,NEAR OCEAN
2271,INLAND


In [31]:
# 카테고리 범주형 값 텍스트 숫자 변환
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[4.],
       [4.],
       [4.],
       [4.],
       [1.],
       [0.],
       [0.],
       [3.],
       [0.],
       [0.]])

In [32]:
# 카테고리 목록
# 범주형 특성마다 카테고리들의 ID 배열 담은 리스트 반환
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [33]:
# 원-핫 인코딩, 범주의 값을 원-핫 벡터로 변환
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
# 출력 결과는 사이파이 희소 행렬
# 0이 아닌 원소 위치만 메모리에 저장
housing_cat_1hot

# 넘파이 배열 변환
housing_cat_1hot.toarray()

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [34]:
cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [35]:
# 간단한 변환기 작성
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

# add_bedrooms_per_room 하이퍼파라미터 => 머신러닝 알고리즘에 도움 여부 확인 가능
# 준비 단계 자동화 => 다양한 조합 자동화 시도 가능, 최상의 조합 탐색, 시간 절약
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs

array([[-117.03, 32.71, 33.0, ..., 'NEAR OCEAN', 5.017656500802568,
        3.691813804173355],
       [-118.16, 33.77, 49.0, ..., 'NEAR OCEAN', 4.473544973544974,
        1.7380952380952381],
       [-120.48, 34.66, 4.0, ..., 'NEAR OCEAN', 5.645833333333333,
        2.7232142857142856],
       ...,
       [-118.38, 34.03, 36.0, ..., '<1H OCEAN', 3.9867172675521823,
        3.332068311195446],
       [-121.96, 37.58, 15.0, ..., '<1H OCEAN', 6.395348837209302,
        3.178890876565295],
       [-122.42, 37.77, 52.0, ..., 'NEAR BAY', 3.4025764895330113,
        2.108695652173913]], dtype=object)

In [36]:
# 변환 파이프라인 => 연속된 변환 순차 처리
# (이름, 추정기) 쌍
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [37]:
# 범주형, 수치형 열과 같은 모든 열 처리
# 전처리 파이프라인 생성 => 각 열에 적절한 변환 시행
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

### 2.6) 모델 선택과 훈련

In [38]:
# 훈련 세트에서 훈련하고 평가하기

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

In [40]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print(f'예측: {lin_reg.predict(some_data_prepared)}')
print(f'레이블: {list(some_labels)}')

예측: [212842.71869744 204346.93995355 207051.22423082 211394.73772039
 208264.22833796]
레이블: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [42]:
# 전체 훈련 세트에 대한 해당 회귀 모델의 RMSE 측정
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

115610.62161702369

In [43]:
# 위의 과소적합 문제 모델 해결 방안
# 좋은 예측 만들만큼 충분한 정보 제공 x or 모델이 충분히 강력 x
# 복잡한 모델 시도

from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

In [44]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse
# 과대 적합

0.0

In [53]:
# 교차 검증을 사용한 평가
from sklearn.model_selection import cross_val_score

# neg_mean_squared_error => mse의 반대 즉 음수값 계산
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                        scoring='neg_mean_squared_error', cv=10)

# 사이킷런 교차검증 => scoring 매개변수에 낮을수록 좋은 비용함수 사용 x
# 클수록 좋은 효용 함수 사용 ㅇ
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([163273.32345609, 163514.49371075, 163413.52655351, 166139.63393959,
       166561.44349161, 166265.00254259, 168745.85032502, 174529.83204151,
       167364.40953398, 162349.2792094 ])

In [56]:
def display_scores(scores):
    print(f'점수: {scores}')
    print(f'평균: {scores.mean()}')
    print(f'표준편차: {scores.std()}')

display_scores(tree_rmse_scores)

점수: [163273.32345609 163514.49371075 163413.52655351 166139.63393959
 166561.44349161 166265.00254259 168745.85032502 174529.83204151
 167364.40953398 162349.2792094 ]
평균: 166215.67948040427
표준편차: 3401.8997943187


In [57]:
# 선형 회귀 모델 점수 계산
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, 
                            scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

# 결정트리 모델이 과대적합되어 선형 회귀 모델보다 성능이 나쁨


점수: [109593.76776239 117338.00887239 115137.60735795 117603.17016568
 115406.28736102 118028.0825645  115080.26407372 117909.30475732
 116970.73889143 114167.65174412]
평균: 115723.48835505196
표준편차: 2421.4907499038613


### 2.7) 모델 세부 튜닝

In [ ]:
# 그리드 탐색
